In [17]:
import os
import cv2
import pandas as pd
import numpy as np

In [18]:
# Paths
path = 'DeepRN'
path_exams = "D:\\Projects\\Datasets"

path_data = "{}\\data.csv".format(path_exams)

path_log = 'log_result.log'

In [19]:
# Params
qt_frames_instantly_before = 8
qt_frames_instantly_after = 8

qt_frames_median_get_size = 5

In [20]:
# Open a data

data = pd.read_csv(path_data)
data.head()

,patient,classify,video,param,stimulus
0,307,Atermo,030703_07_2019_08_09_08,"RGBI= {78,45,40,99}","[{'stimul': 'RED', 'init_stimul': 68, 'end_sti..."
1,307,Atermo,030703_07_2019_08_10_26,"RGBI= {78,45,40,99}","[{'stimul': 'BLUE', 'init_stimul': 30, 'end_st..."
2,307,Atermo,030703_07_2019_08_15_23,"RGBI= {78,45,40,99}","[{'stimul': 'BLUE', 'init_stimul': 57, 'end_st..."
3,307,Atermo,030703_07_2019_08_16_37,"RGBI= {78,45,40,99}","[{'stimul': 'RED', 'init_stimul': 42, 'end_sti..."
4,3073,Retina Imatura,0307303_07_2019_08_21_00,"RGBI= {78,45,40,99}","[{'stimul': 'GREEN', 'init_stimul': 24, 'end_s..."


In [21]:
def get_fps_video(video):
    exam_path = "{}\\Exams\\{}.mp4".format(path_exams, video)
    exam = cv2.VideoCapture(exam_path)
    return exam.get(cv2.CAP_PROP_FPS)

In [22]:
def load_label_data(video):
    label_path = "{}\\{}_label.csv".format(path, video)
    return pd.read_csv(label_path)

In [23]:
def get_list_stimulus(stimulus):
    aux = stimulus.replace('[', '').replace(']', '')
    list_stimuls = aux.split('}, {')
    
    if len(list_stimuls) == 0:
        raise Exception('Falha por nao ter estimulos')
        
    return list_stimuls

In [24]:
def describe_stimul(stimul):
    stimul = stimul.replace('{', '').replace('}', '')

    # Get a color stimul
    stimul_color = stimul[stimul.find(':')+2:stimul.find(",")]
    stimul_color = stimul_color.replace("'", '')
    stimul = stimul[stimul.find(",")+2:]

    # Get a init frame stimul
    init_stimul = int(stimul[stimul.find(':')+2:stimul.find(",")])
    stimul = stimul[stimul.find(",")+2:]

    # Get a end frame stimul
    end_stimul = int(stimul[stimul.find(':')+2:len(stimul)])
    
    return stimul_color, init_stimul, end_stimul

In [25]:
def get_size_stimul(label, number_frame):
    range_min = number_frame - qt_frames_median_get_size
    range_min = range_min if range_min > 0 else 0
    
    range_max = number_frame + qt_frames_median_get_size + 1
    range_max = range_max if range_max < label.shape[0] else label.shape[0]-1
    
    list_sizes = np.array([])
    for i in range(range_min, range_max):
        list_sizes = np.append(list_sizes, label['radius'][i])    

    list_sizes.sort()
    list_sizes = list_sizes[1:len(list_sizes)-1]
    
    return np.mean(list_sizes)

In [26]:
def get_sizes_instantly_after_before(label, init_stimul, end_stimul):
    frame_instantly_before = init_stimul - qt_frames_instantly_before
    frame_instantly_after = end_stimul + qt_frames_instantly_after

    size_instantly_before = get_size_stimul(label, frame_instantly_before)
    size_instantly_after = get_size_stimul(label, frame_instantly_after)    
    
    return size_instantly_before, size_instantly_after

In [27]:
def normalize(x):
    x = np.asarray(x)
    y = (x - x.min()) / (np.ptp(x))
    return y * x.max()

In [28]:
def save_log(information):
    with open(path_log, 'a') as file_writer:
        file_writer.write('{}\n'.format(information))

In [29]:
def get_min_max_values_range(label, init_range, end_range):
    list_sizes = np.array([])
    
    init_range = init_range if init_range > 0 else 0
    end_range = end_range if end_range < label.shape[0]-1 else label.shape[0]-1
    
    save_log('func get_min_max init_range: {} end_range: {}'.format(init_range, end_range))
    
    for i in range(init_range, end_range-1):
        list_sizes = np.append(list_sizes, label['radius'][i])
    
    list_sizes.sort()
    return list_sizes[0:3], list_sizes[len(list_sizes)-3: len(list_sizes)]

In [30]:
data_final = pd.DataFrame({'patient': [], 'classify': [], 'video': [], 'param': [], 'stimul': [],
                           '3_seconds_before': [],
                           'size_instantly_before_stimul': [],
                           'size_instantly_after_stimul': [],
                           '3_seconds_after': [],
                           '5_seconds_after': [],
                           '6_seconds_after': [],
                           '10_seconds_after': [],
                           '15_seconds_after': [],
                           'min_value1': [],
                           'min_value2': [],
                           'min_value3': [],
                           'max_value1': [],
                           'max_value2': [],
                           'max_value3': []})

for index, values in data.iterrows():    
    # Get a patient informations
    video = values['video']    
    patient = str(values['patient'])
    classify = str(values['classify'])
    param = str(values['param'])
    stimulus = str(values['stimulus'])
    
    # Get a FPS Exam
    fps = get_fps_video(video)
    
    # Get a labels of video
    label = load_label_data(video)

    if label.shape[0] <= 0:
        continue
    
    # Get a list of stimulus
    list_stimuls = get_list_stimulus(stimulus)    
    
    save_log('Exam: {}'.format(video))
    for i in range(0, len(list_stimuls)):
        # get information stimul
        stimul_color, init_stimul, end_stimul = describe_stimul(list_stimuls[i])
        save_log('stimul_color: {}, init_stimul: {}, end_stimul: {}'.format(stimul_color, init_stimul, end_stimul))
        
        # get size instantly before and after stimul
        size_instantly_before, size_instantly_after = get_sizes_instantly_after_before(label, init_stimul, end_stimul)
        
        # get sizes before and after stimul
        frame_3_seconds_before = init_stimul - int(fps * 3)
        
        frame_3_seconds_after = end_stimul + int(fps * 3)
        frame_5_seconds_after =  end_stimul + int(fps * 5)
        frame_6_seconds_after = end_stimul + int(fps * 6)
        frame_10_seconds_after = end_stimul + int(fps * 10)
        frame_15_seconds_after = end_stimul + int(fps * 15)

        save_log("frame_3_before: {}, frame_3_after: {}, frame_5_after: {}, frame_6_after: {} frame_10_after: {}, frame_15_ater: {}".format(frame_3_seconds_before, frame_3_seconds_after, frame_5_seconds_after, frame_6_seconds_after, frame_10_seconds_after, frame_15_seconds_after))
        
        # valid number frames and get size stimuls
        size_3_seconds_before = get_size_stimul(label, frame_3_seconds_before)
                
        if i < len(list_stimuls)-1:
            _, limit_max, _ = describe_stimul(list_stimuls[i+1])            
        else:
            limit_max = label.shape[0]
            
        size_3_seconds_after = get_size_stimul(
            label, frame_3_seconds_after) if frame_3_seconds_after < limit_max else np.NaN

        size_5_seconds_after = get_size_stimul(
            label, frame_5_seconds_after) if frame_5_seconds_after < limit_max else np.NaN

        size_6_seconds_after = get_size_stimul(
            label, frame_6_seconds_after) if frame_6_seconds_after < limit_max else np.NaN

        size_10_seconds_after = get_size_stimul(
            label, frame_10_seconds_after) if frame_10_seconds_after < limit_max else np.NaN

        size_15_seconds_after = get_size_stimul(
            label, frame_15_seconds_after) if frame_15_seconds_after < limit_max else np.NaN
        
        # Get a max values between end stimul and 3 seconds end stimul range
        _, max_value = get_min_max_values_range(label, end_stimul, limit_max)
        
        # Get a min values between 3 seconds before stimul and init stimul
        min_value, _ = get_min_max_values_range(label, frame_3_seconds_before, init_stimul)
        
        # Add register final data
        data_final = data_final.append(pd.DataFrame({'patient': [patient], 'classify': [classify], 'video': [video],
                                                     'param': [param], 'stimul': [stimul_color],
                                                     '3_seconds_before': [size_3_seconds_before],
                                                     'size_instantly_before_stimul': [size_instantly_before],
                                                     'size_instantly_after_stimul': [size_instantly_after],
                                                     '3_seconds_after': [size_3_seconds_after],
                                                     '5_seconds_after': [size_5_seconds_after],
                                                     '6_seconds_after': [size_6_seconds_after],
                                                     '10_seconds_after': [size_10_seconds_after],
                                                     '15_seconds_after': [size_15_seconds_after],
                                                     'min_value1': [min_value[0]],
                                                     'min_value2': [min_value[1]],
                                                     'min_value3': [min_value[2]],
                                                     'max_value1': [max_value[0]],
                                                     'max_value2': [max_value[1]],
                                                     'max_value3': [max_value[2]]}))

print(data_final)

C:\Users\marco\anaconda3\envs\Classify\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\marco\anaconda3\envs\Classify\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


   patient         classify                      video                param  \
0      307           Atermo    030703_07_2019_08_09_08  RGBI= {78,45,40,99}   
0      307           Atermo    030703_07_2019_08_09_08  RGBI= {78,45,40,99}   
0      307           Atermo    030703_07_2019_08_09_08  RGBI= {78,45,40,99}   
0      307           Atermo    030703_07_2019_08_09_08  RGBI= {78,45,40,99}   
0      307           Atermo    030703_07_2019_08_09_08  RGBI= {78,45,40,99}   
..     ...              ...                        ...                  ...   
0   310701   ROP - Alterado  31070131_07_2019_08_06_31  RGBI= {78,45,40,90}   
0   310702  Catarata ou ROP  31070231_07_2019_08_11_10  RGBI= {78,45,40,90}   
0   310702  Catarata ou ROP  31070231_07_2019_08_11_10  RGBI= {78,45,40,90}   
0   310702  Catarata ou ROP  31070231_07_2019_08_12_46  RGBI= {78,45,40,90}   
0   310702  Catarata ou ROP  31070231_07_2019_08_12_46  RGBI= {78,45,40,90}   

   stimul  3_seconds_before  size_instantly_before_

In [31]:
data_final.to_csv('data_final.csv', index=False)